In [4]:
import numpy as np 
from train_nvidia import AutoEncoder, get_dataloaders, RMSE, write_submission
import torch 

/home/ubuntu/anaconda3/envs/rf39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
item_based = True 
dropout = 0.5
input_size = 10000 if item_based else 1000
layer_sizes = [input_size,128,128,128,128]

In [11]:
path = '/home/ubuntu/projects/CILProject22/reports/logs/20220727-103246_ItemEncoder_bs64_lr0.001_wd0.01_do0.5_ls[10000, 128, 128, 128, 128]_submit_unb_True_0.99983'

In [12]:
for i in range(5):
    model = AutoEncoder(
          layer_sizes=layer_sizes, 
          nl_type='selu', 
          is_constrained=True, 
          dp_drop_prob=dropout, 
          last_layer_activations=True
          )
    model.load_state_dict(torch.load(path + f'/UserAutoEncoder_split_{i}' + f"/best_model.pt"))
    train_loader, valid_loader, data_matrix, valid_mask = get_dataloaders('./data/raw' + f"/5fold", split_number=i, batch_size=100, item_based=item_based)
    # run inference on data matrix 
    model.eval()
    model = model.cuda()
    avg_val = [] 
    with torch.no_grad():
        z = model(data_matrix.cuda())
        model_loss = RMSE(z, data_matrix.cuda(), valid_mask.cuda())
        avg_val.append(model_loss.item())
        print(f'Model {i} RMSE: {model_loss.item()}')
        z = z.cpu().numpy()
        # write validation submission 
        write_submission(z, f"./data/raw/5fold/test_split_{i}.csv", f"{path}/UserAutoEncoder_split_{i}/valid_submission.csv", item_based=item_based)
print(f"---\nAverage validation RMSE: {np.mean(avg_val)}")

Model 0 RMSE: 1.0007834434509277
Model 1 RMSE: 0.9989818930625916
Model 2 RMSE: 1.0005303621292114
Model 3 RMSE: 0.9980558753013611
Model 4 RMSE: 0.999833345413208
---
Average validation RMSE: 0.999833345413208
